#### 1) Прочитать статьи про BPR, WARP loss

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией
    Подбор параметров:
    - Loss
    - no_components
    - regularization

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

G:\Anaconda\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [9]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [10]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [12]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\lufta\Documents\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


In [13]:

user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [15]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [16]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [18]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [60]:
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [63]:
def init_model(component, los, regular, item, user):
    model = LightFM(no_components=component,
                    loss=los,
                    learning_rate=regular, 
                    item_alpha=item,
                    user_alpha=user, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)       
    return model

def fit_model(model):
    return model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
            sample_weight=coo_matrix(user_item_matrix),
            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
            epochs=20, 
            num_threads=20)       

def predict_model(model):
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    return predictions


In [93]:
import time

params = {
    'loss':['bpr', 'warp'],
    'no_components':[10, 25, 50, 100, 150],
    'learning_rate':[0.01, 0.05, 0.1],
    'item_alpha':[0.3, 0.4],
    'user_alpha':[0.1, 0.2]
}


result_feat = {
    'loss':[],
    'no_components':[],
    'learning_rate':[],
    'item_alpha':[],
    'user_alpha':[],
    'precision_train':[],
    'precision_test':[]
}

def grid_search(params):
    count = 1
    for i in params.values():
        count *= len(i)

    start_time = time.time()
    i = 0
    for component in params['no_components']:
        for los in params['loss']:
            for regular in params['learning_rate']: 
                for item in params['item_alpha']: 
                    for user in params['user_alpha']: 
                        result_feat['loss'].append(los)
                        result_feat['no_components'].append(component)
                        result_feat['learning_rate'].append(regular)
                        result_feat['item_alpha'].append(item)
                        result_feat['user_alpha'].append(user)

                        model = init_model(component, los, regular, item, user)
                        fit_model(model)

                        train_precision = precision_at_k(model, sparse_user_item, 
                                         user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                         item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                         k=5).mean()

                        predictions = predict_model(model)

                        data_train_filtered['score'] = predictions
                        predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
                                            unique().reset_index()
                        df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
                        precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()

                        result_feat['precision_train'].append(train_precision)
                        result_feat['precision_test'].append(precision)

                        i +=1
                        total_time = time.time()

                        if i % 10 == 0 or i == 1 or i == count:                
                            print(f'Итерация {i} из {count}. Времени затрачено: {((total_time - start_time) / 60):.2f} минут.')
    return pd.DataFrame(result_feat)

In [94]:
grid_search(params)

Итерация 1 из 120. Времени затрачено: 0.71 минут.
Итерация 10 из 120. Времени затрачено: 6.24 минут.
Итерация 20 из 120. Времени затрачено: 13.09 минут.
Итерация 30 из 120. Времени затрачено: 24.87 минут.
Итерация 40 из 120. Времени затрачено: 37.53 минут.
Итерация 50 из 120. Времени затрачено: 56.45 минут.
Итерация 60 из 120. Времени затрачено: 88.58 минут.
Итерация 70 из 120. Времени затрачено: 126.39 минут.
Итерация 80 из 120. Времени затрачено: 188.36 минут.
Итерация 90 из 120. Времени затрачено: 255.42 минут.
Итерация 100 из 120. Времени затрачено: 347.25 минут.
Итерация 110 из 120. Времени затрачено: 451.83 минут.
Итерация 120 из 120. Времени затрачено: 578.67 минут.


,loss,no_components,learning_rate,item_alpha,user_alpha,precision_train,precision_test
0,bpr,10,0.01,0.3,0.1,0.438526,0.143087
1,bpr,10,0.01,0.3,0.2,0.200401,0.142496
2,bpr,10,0.01,0.4,0.1,0.438526,0.142299
3,bpr,10,0.01,0.4,0.2,0.201362,0.139245
4,bpr,10,0.05,0.3,0.1,0.206968,0.065353
...,...,...,...,...,...,...,...
115,warp,150,0.05,0.4,0.2,0.221145,0.082200
116,warp,150,0.10,0.3,0.1,0.211534,0.068900
117,warp,150,0.10,0.3,0.2,0.211053,0.068801
118,warp,150,0.10,0.4,0.1,0.209451,0.069491


In [95]:
param_result = pd.DataFrame(result_feat)
param_result.sort_values(by=['precision_test'], ascending=False).head(20)

,loss,no_components,learning_rate,item_alpha,user_alpha,precision_train,precision_test
108,warp,150,0.01,0.3,0.1,0.294113,0.145057
84,warp,100,0.01,0.3,0.1,0.294113,0.143580
26,bpr,25,0.01,0.4,0.1,0.438526,0.143383
0,bpr,10,0.01,0.3,0.1,0.438526,0.143087
24,bpr,25,0.01,0.3,0.1,0.438526,0.143087
29,bpr,25,0.05,0.3,0.2,0.206488,0.142890
52,bpr,50,0.05,0.3,0.1,0.200561,0.142890
18,warp,10,0.05,0.4,0.1,0.294113,0.142693
62,warp,50,0.01,0.4,0.1,0.294113,0.142594
72,bpr,100,0.01,0.3,0.1,0.438526,0.142594


In [97]:
model = init_model(25, 'bpr', 0.01, 0.4, 0.1)
fit_model(model)

train_precision = precision_at_k(model, sparse_user_item, 
                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                 k=5).mean()

predictions = predict_model(model)

In [98]:
data_train_filtered['score'] = predictions
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
                    unique().reset_index()
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [100]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.14338259441707532
